# sf testing book

> use this to test salesforce.py

In [ ]:
##| default_exp test_sf_1

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import salesforce as SF
# import salesforce_old as SF
import doubledot.crema_sf as SF
from doubledot.ATMS_api import ATMS_api
import jmespath as jp
import json
import pandas as pd
from nbdev.showdoc import *
from fastcore.basics import patch
    
from fastcore.test import test_eq

In [ ]:
# test_eq("dog","cat")

In [ ]:
sf = SF.Salesforce()

FileNotFoundError: [Errno 2] No such file or directory: 'secrets.json'

In [ ]:
sf.sf_access_token

In [ ]:
sf.test_token()

In [ ]:
sf.get_sf_object_ids()

In [ ]:
atms = ATMS_api()

In [ ]:
atms.telus_access_token

## Get ATMS Objects

In [ ]:
# get memberships data
atms.write_obj_to_file('memberships',initial_offset=59_000, max_rows=500)
atms.write_obj_to_file('contacts', initial_offset=450_000, max_rows=500)
atms.write_obj_to_file('items')


# atms.write_obj_to_file('sales', initial_offset=450_000, max_rows=500)

## clean ATMS data and load into atms.obj_d 

In [ ]:
atms.clean_data_file('contacts')
atms.load_data_file_to_dict('contacts')
print(atms.obj_d.keys())
# atms.obj_d['contacts'][:3]

In [ ]:
atms.clean_data_file('memberships')
atms.load_data_file_to_dict('memberships')
print(atms.obj_d.keys())
# atms.obj_d['memberships'][:3]

In [ ]:
"""[
  {
    "membershipId": 0,
    "memberSince": "2023-05-02T21:47:29.055Z",
    "updateDate": "2023-05-02T21:47:29.055Z",
    "membershipTerms": [
      {
        "membershipTermId": 0,
        "membershipKey": 0,
        "effectiveDate": "2023-05-02T21:47:29.055Z",
        "expiryDate": "2023-05-02T21:47:29.055Z",
        "membershipType": "string",
        "upgradeFromTermKey": 0,
        "giftMembership": true,
        "refunded": true,
        "saleDetailKey": 0,
        "itemKey": 0,
        "membershipMembers": [
          {
            "membershipMemberId": 0,
            "membershipTermKey": 0,
            "cardNumber": 0,
            "membershipNumber": "string",
            "cardStatus": "string",
            "contactKey": 0,
            "displayName": "string"
          }
        ]
      }
    ]
  }
]
"""

## create ``Memberships[]`` csv file

In [ ]:
## requires that atms.obj_d['memberships'] is loaded
## we make this for bulk insert into salesforce
assert atms.obj_d['memberships']

mem_s = "[].{membershipId__c: membershipId, \
    memberSince__c: memberSince, \
    updateDate__c: updateDate}"

# make membership members table
dict_l = jp.search(mem_s, atms.obj_d['memberships'])
with open('memberships.csv', 'w') as f:
    f.write('\t'.join(dict_l[0].keys()) + '\n') # header
    for d in dict_l:
        f.write('\t'.join([str(v) for v in d.values()]) + '\n')

## create ``Memberships[].membershipTerms`` csv file

In [ ]:
## requires that atms.obj_d['memberships'] is loaded
## we make this for bulk insert into salesforce

# make membership term table
# we use a hack of inserting _1_ into the membershipKey__r field name to get around the fact that jmespath won't allow a dot in a field name
assert atms.obj_d['memberships']

memTerm_s = "[:50].membershipTerms[].{membershipTermId__c: membershipTermId,\
membershipKey__r_1_membershipId__c:membershipKey,\
effectiveDate__c:effectiveDate,\
expiryDate__c:expiryDate,\
membershipType__c:membershipType,\
upgradeFromTermKey__c:upgradeFromTermKey,\
giftMembership__c:giftMembership,\
refunded__c:refunded,\
saleDetailKey__c:saleDetailKey,\
itemKey__c:itemKey}"
dict_l = jp.search(memTerm_s, atms.obj_d['memberships'])
with open('membership_terms.csv', 'w') as f:
    # hack to create header with a dot in it, jmespath won't do it
    f.write('\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()]) + '\n') # header
    for d in dict_l:
        #changed this to not write None for empty values, eg "" for null and false (a default value)
        f.write('\t'.join([str(v) if v else "" for v in d.values()]) + '\n')

In [ ]:
with open('test.json', 'w') as f:
    f.write(json.dumps(dict_l[:5], indent=4))

In [ ]:
with open('test.json', 'r') as f:
    test = json.load(f)


## create ``Memberships[].membershipTerms[].membershipMembers[]`` csv file

In [ ]:
## requires that atms.obj_d['memberships'] is loaded
## we make this for bulk insert into salesforce

# make membership term table
# we use a hack of inserting _1_ into the membershipKey__r field name to get around the fact that jmespath won't allow a dot in a field name
try: # don't want to instantiate  this more than I have 
    atms
except NameError:
    atms = ATMS_api()
finally:
    try:
        print("trying")
        obj_s = 'memberships'
        atms.write_obj_to_file(obj_s, max_rows=500)
        atms.load_data_file_to_dict(obj_s)
        assert obj_s in atms.obj_d, f" {obj_s} not in {atms.obj_d.keys()}"
        atms.load_data_file_to_dict('non-existant file') #this should throw an erro
    except FileNotFoundError:
        print("There is no dirty file, we quit")

memMembers_s = "[].membershipTerms[].membershipMembers[].{membershipMemberId__c:membershipMemberId,\
membershipTermKey__r_1_membershipTermId__c:membershipTermKey,\
cardNumber__c:cardNumber,\
membershipNumber__c:membershipNumber,\
cardStatus__c:cardStatus,\
contactKey__c:contactKey,\
displayName__c:displayName}"

# make membership members table
dict_l = jp.search(memMembers_s, atms.obj_d['memberships'])
with open('membership_members.csv', 'w') as f:
    f.write('\t'.join([s.replace('_1_','.') for s in dict_l[0].keys()]) + '\n') # header
    # f.write('\t'.join(dict_l[0].keys()) + '\n') # header
    for d in dict_l:
        f.write('\t'.join([str(v) for v in d.values()]) + '\n')

## make dataframes for membership, membership terms, and membership members, and contacts

In [ ]:
members_df = pd.read_csv('membership_members.csv', sep='\t') 
membershipTerms_df = pd.read_csv('membership_terms.csv', sep='\t')
membership_df = pd.read_csv('memberships.csv', sep='\t')

atms.load_data_file_to_dict(obj_s='contacts')
contacts_df = pd.DataFrame(atms.obj_d['contacts'])
contacts_df.dtypes

### merge membership and membershipTerms

In [ ]:
membership_df.merge(membershipTerms_df, how='right', left_on='membershipId__c', right_on='membershipKey__r.membershipId__c')

### trying to solve problem of unmatched contacts and members usin dataframes

In [ ]:
contacts_df.contactId.max(),contacts_df.contactId.min(),members_df.loc[:,"contactKey__c"].min(),members_df.loc[:,"contactKey__c"].max(),   

### intersection of contactIds and members.contactKey__c

In [ ]:
s1 = set(contacts_df.contactId) 
s2 = set(members_df.loc[:,"contactKey__c"])
# common elements of s1 and s2
s1.intersection(s2) 

In [ ]:
_df = members_df
_df.merge(contacts_df, left_on='contactKey__c', right_on ='contactId', how='inner')#.loc[:, contacts_df.columns]
# _df.dtypes

In [ ]:

print('\n'.join(set(jp.search("[].membershipTerms[].membershipMembers[].cardStatus", atms.obj_d['memberships']))))

In [ ]:
# finde unique membershipTerm membershipTypes 
print('\n'.join(set(jp.search("[].membershipTerms[].membershipType", atms.obj_d['memberships']))))

In [ ]:
# list of unique upgrade from Term Key  
print('\n'.join({str(i) for i in jp.search("[].membershipTerms[].upgradeFromTermKey", atms.obj_d['memberships'])}))

## upload Memberships to SF 

In [ ]:
sf.create_job('Membership__c') # job_id now in sf.job_id
sf.bulk_job_id

sf.upload_csv('memberships.csv')
    
sf.close_job()
sf.job_status()
sf.failed_results()

## upload MembershipTerm to SF 

In [ ]:
sf = SF.Salesforce()
sf.create_job('MembershipTerm__c')
sf.bulk_job_id

sf.upload_csv('membership_terms.csv')
    
sf.close_job()
sf.failed_results()
sf.successful_results()
sf.job_status()
# sf.delete_sf_objects('membershipTerm__c')

## upload MembershipMembers to SF 

In [ ]:
atms = ATMS_api()

In [ ]:
## generate tests for all the methods

## upload a file of contacts to salesforce
## verify that all the contacts are in salesforce
## remove all contacts from salesforce


In [ ]:
_d = {
  "saleKey": "string",
  "saleAmount": "string",
  "paymentAmount": "string",
  "saleDate": "2023-05-05T00:06:52.646Z",
  "active": True,
  "terminalKey": 0,
  "ticketCount": 0,
  "eventDate": "2023-05-05T00:06:52.646Z",
  "booking": {
    "bookingId": 0,
    "bookingContactKey": 0,
    "bookingContactType": "string",
    "contactKey": 0,
    "contactIndividualKey": 0,
    "contactOrganizationKey": 0,
    "displayName": "string",
    "firstName": "string",
    "lastName": "string",
    "email": "string",
    "phone": "string"
  },
  "saleComment": "string",
  "saleDetails": [
    {
      "saleDetailId": 0,
      "saleId": 0,
      "itemKey": 0,
      "scheduleKey": 0,
      "rateKey": 0,
      "categoryKey": 0,
      "itemQuantity": 0,
      "pricingPriceKey": 0,
      "itemPrice": 0,
      "itemTotal": 0,
      "couponTotal": 0,
      "discountTotal": 0,
      "total": 0,
      "revenueDate": "2023-05-05T00:06:52.646Z",
      "refundReasonKey": 0,
      "systemPriceOverride": "string",
      "membershipTermKey": 0,
      "taxTotal": 0,
      "surchargeTotal": 0,
      "surchargeTaxTotal": 0,
      "pendingShiftKey": 0,
      "pendingTerminalKey": 0,
      "pendingDateTimeStamp": "2023-05-05T00:06:52.646Z",
      "pendingUser": "string",
      "finalStatus": 0,
      "finalShiftKey": 0,
      "finalTerminalKey": 0,
      "finalDateTimeStamp": "2023-05-05T00:06:52.646Z",
      "finalUser": "string",
      "refundQuantity": 0,
      "firstScheduleDetailKey": 0,
      "itemDescription": "string",
      "rateDescription": "string",
      "categoryDescription": "string",
      "scheduleDate": "2023-05-05T00:06:52.646Z"
    }
  ],
  "tickets": [
    {
      "ticketKey": 0,
      "saleKey": 0,
      "saleDetailKey": 0,
      "itemKey": 0,
      "itemDescription": "string",
      "itemShort": "string",
      "itemType": "string",
      "scheduleKey": 0,
      "scheduleDate": "2023-05-05T00:06:52.646Z",
      "facilityKey": 0,
      "facilityDescription": "string",
      "facilityShort": "string",
      "ticketText": "string",
      "rate": "string",
      "category": "string",
      "cancelled": "2023-05-05T00:06:52.646Z",
      "redeemed": "2023-05-05T00:06:52.646Z",
      "refunded": "2023-05-05T00:06:52.646Z",
      "isValid": True,
      "validationText": "string",
      "redeemedCount": 0,
      "ticketDetailKey": 0,
      "scheduleDetailKey": 0,
      "seats": 0,
      "seatsSold": 0,
      "totalTicketsScanned": 0,
      "scheduleEndDate": "2023-05-05T00:06:52.646Z",
      "ticketStatus": "string",
      "ticketDisplay": "string",
      "ticketSection": "string",
      "ticketRow": "string",
      "ticketSeat": "string",
      "ticketPrefix": "string",
      "active": True,
      "cancelable": True,
      "contactIndividualKey": 0,
      "individualKey": 0,
      "individualName": "string",
      "contactOrganizationKey": 0,
      "organizationKey": 0,
      "organizationName": "string",
      "reschedulable": True,
      "rescheduleLead": 0,
      "rescheduleGrace": 0,
      "rescheduleCount": 0,
      "rescheduleLimitHours": 0,
      "comment": "string",
      "transferRequestKey": 0,
      "recipientEmail": "string"
    }
  ]
}

jp.search("keys(@)",_d)